In [1]:
using Laplacians

objc[23944]: Class TKApplication is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[23944]: Class TKMenu is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[23944]: Class TKContentView is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[23944]: Class TKWindow is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.


In [2]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgWorker (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
graphp = "../graphs/se_3/"

"../graphs/se_3/"

In [5]:
a = readFromFile(graphp * "graph.mtx");

In [6]:
tree = readFromFile(graphp * "tree.mtx");

In [7]:
x = readFromFile(graphp * "x.vec");

In [8]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [9]:
@time F = treeSolver(tree)
@time F(b);

  8.595874 seconds (2.81 M allocations: 184.029 MB, 0.81% gc time)
  0.290550 seconds (4.12 M allocations: 81.314 MB, 20.11% gc time)


In [10]:
# a run on standard Float64 data types
numIts = 1000;
@time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)
insert!(dbg, 1, "$(numIts) iterations")
insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
writeToFile(graphp * "_log_julia_default.txt", dbg)

Working on iteration 10 with a-norm error 1.7712969042254374e6
Working on iteration 20 with a-norm error 1.5187944309197352e6
Working on iteration 30 with a-norm error 914283.1966626841
Working on iteration 40 with a-norm error 600897.3539355926
Working on iteration 50 with a-norm error 415369.3301646716
Working on iteration 60 with a-norm error 288920.0490711332
Working on iteration 70 with a-norm error 216874.4499564585
Working on iteration 80 with a-norm error 175046.37152892043
Working on iteration 90 with a-norm error 140263.3578234383
Working on iteration 100 with a-norm error 122414.5937553661
Working on iteration 110 with a-norm error 101754.96776614258
Working on iteration 120 with a-norm error 86652.12751049017
Working on iteration 130 with a-norm error 74546.99470999758
Working on iteration 140 with a-norm error 67401.32871224993
Working on iteration 150 with a-norm error 60766.59981879617
Working on iteration 160 with a-norm error 52630.42938957592
Working on iteration 170 

In [11]:
numIts = 1000
for hp in [64,128,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    FHP = treeSolver(treeHP)
    
    @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_$(hp).txt", dbg)
end

Working on iteration 10 with a-norm error 1.7661444089471698e6
Working on iteration 20 with a-norm error 1.4452984951163784e6
Working on iteration 30 with a-norm error 808680.5588834081
Working on iteration 40 with a-norm error 501271.91371562955
Working on iteration 50 with a-norm error 321162.4750302397
Working on iteration 60 with a-norm error 228074.6799846653
Working on iteration 70 with a-norm error 169555.30135046845
Working on iteration 80 with a-norm error 136012.03698534184
Working on iteration 90 with a-norm error 115307.31198349627
Working on iteration 100 with a-norm error 89570.74681898285
Working on iteration 110 with a-norm error 79344.37276350394
Working on iteration 120 with a-norm error 68068.06360335698
Working on iteration 130 with a-norm error 59168.665940437364
Working on iteration 140 with a-norm error 53203.879482011136
Working on iteration 150 with a-norm error 46763.61846356175
Working on iteration 160 with a-norm error 42395.97772838519
Working on iteration 